<a href="https://colab.research.google.com/github/Namra-3305/PaperSense-Intelligent-Document-Platform/blob/main/src/data_processing/cnn_dailymail_preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
# # src/data_processing/cnn_dailymail_preprocessing.py
# """
# CNN/DailyMail Dataset Preprocessing
# Handles text summarization data preparation
# """

# import os
# import json
# import pandas as pd
# import numpy as np
# import torch
# from datasets import load_dataset, load_from_disk
# import matplotlib.pyplot as plt
# import seaborn as sns
# from transformers import AutoTokenizer
# import nltk
# from nltk.tokenize import sent_tokenize, word_tokenize
# from nltk.corpus import stopwords
# import re
# from collections import Counter
# from tqdm import tqdm
# import pickle

# # Download required NLTK data
# nltk.download('punkt_tab', quiet=True)
# nltk.download('stopwords', quiet=True)

# # Project paths - FIXED
# PROJECT_ROOT = "/content/drive/MyDrive/PaperSense-Intelligent-Document-Platform"
# RAW_DATA_DIR = os.path.join(PROJECT_ROOT, "datasets", "raw", "cnn_dailymail")
# PROCESSED_DATA_DIR = os.path.join(PROJECT_ROOT, "datasets", "processed", "cnn_dailymail")
# RESULTS_DIR = os.path.join(PROJECT_ROOT, "results", "visualizations", "generation")

# def ensure_dir(path):
#     """Create directory if it doesn't exist"""
#     os.makedirs(path, exist_ok=True)
#     return path

# def download_cnn_dailymail_dataset():
#     """Download and save CNN/DailyMail dataset"""
#     print("📥 Downloading CNN/DailyMail dataset...")

#     try:
#         # Download dataset from Hugging Face
#         dataset = load_dataset("cnn_dailymail", "3.0.0")

#         # Create directory and save
#         ensure_dir(os.path.dirname(RAW_DATA_DIR))
#         dataset.save_to_disk(RAW_DATA_DIR)

#         print(f"✅ Dataset downloaded and saved to: {RAW_DATA_DIR}")
#         return dataset
#     except Exception as e:
#         print(f"❌ Error downloading dataset: {e}")
#         return None

# def load_cnn_dailymail_dataset():
#     """Load CNN/DailyMail dataset from disk or download if not exists"""
#     print("📂 Loading CNN/DailyMail dataset...")

#     try:
#         # Try to load from disk first
#         if os.path.exists(RAW_DATA_DIR) and os.listdir(RAW_DATA_DIR):
#             dataset = load_from_disk(RAW_DATA_DIR)
#             print(f"✅ Dataset loaded from disk with splits: {list(dataset.keys())}")
#         else:
#             print("📥 Dataset not found on disk, downloading...")
#             dataset = download_cnn_dailymail_dataset()
#             if dataset is None:
#                 return None

#         # Print basic info
#         for split_name, split_data in dataset.items():
#             print(f"  {split_name}: {len(split_data)} samples")

#         return dataset
#     except Exception as e:
#         print(f"❌ Error loading dataset: {e}")
#         print("🔄 Attempting to download dataset...")
#         return download_cnn_dailymail_dataset()

# def analyze_text_statistics(dataset):
#     """Analyze text length and content statistics"""
#     print("📊 Analyzing text statistics...")

#     ensure_dir(RESULTS_DIR)

#     # Collect statistics
#     article_lengths = []
#     summary_lengths = []
#     article_sent_counts = []
#     summary_sent_counts = []

#     # Sample from train set for analysis
#     train_data = dataset['train']
#     sample_size = min(10000, len(train_data))  # Sample for faster analysis

#     # Fix: Convert numpy integers to Python integers
#     sample_indices = np.random.choice(len(train_data), sample_size, replace=False)
#     sample_indices = [int(idx) for idx in sample_indices]  # Convert to Python int

#     print(f"Analyzing {sample_size} samples from training set...")

#     for idx in tqdm(sample_indices, desc="Analyzing samples"):
#         article = train_data[idx]['article']
#         summary = train_data[idx]['highlights']

#         # Word counts
#         article_words = len(word_tokenize(article))
#         summary_words = len(word_tokenize(summary))
#         article_lengths.append(article_words)
#         summary_lengths.append(summary_words)

#         # Sentence counts
#         article_sents = len(sent_tokenize(article))
#         summary_sents = len(sent_tokenize(summary))
#         article_sent_counts.append(article_sents)
#         summary_sent_counts.append(summary_sents)

#     # Create visualizations
#     fig, axes = plt.subplots(2, 3, figsize=(18, 10))

#     # Article length distribution
#     axes[0,0].hist(article_lengths, bins=50, alpha=0.7, color='blue', edgecolor='black')
#     axes[0,0].set_title('Article Length Distribution (Words)')
#     axes[0,0].set_xlabel('Number of Words')
#     axes[0,0].set_ylabel('Frequency')
#     axes[0,0].axvline(np.mean(article_lengths), color='red', linestyle='--',
#                      label=f'Mean: {np.mean(article_lengths):.0f}')
#     axes[0,0].legend()

#     # Summary length distribution
#     axes[0,1].hist(summary_lengths, bins=50, alpha=0.7, color='green', edgecolor='black')
#     axes[0,1].set_title('Summary Length Distribution (Words)')
#     axes[0,1].set_xlabel('Number of Words')
#     axes[0,1].set_ylabel('Frequency')
#     axes[0,1].axvline(np.mean(summary_lengths), color='red', linestyle='--',
#                      label=f'Mean: {np.mean(summary_lengths):.0f}')
#     axes[0,1].legend()

#     # Length ratio distribution
#     ratios = [s/a if a > 0 else 0 for a, s in zip(article_lengths, summary_lengths)]
#     axes[0,2].hist(ratios, bins=50, alpha=0.7, color='orange', edgecolor='black')
#     axes[0,2].set_title('Summary/Article Length Ratio')
#     axes[0,2].set_xlabel('Ratio')
#     axes[0,2].set_ylabel('Frequency')
#     axes[0,2].axvline(np.mean(ratios), color='red', linestyle='--',
#                      label=f'Mean: {np.mean(ratios):.3f}')
#     axes[0,2].legend()

#     # Article sentence count
#     axes[1,0].hist(article_sent_counts, bins=30, alpha=0.7, color='purple', edgecolor='black')
#     axes[1,0].set_title('Article Sentence Count Distribution')
#     axes[1,0].set_xlabel('Number of Sentences')
#     axes[1,0].set_ylabel('Frequency')
#     axes[1,0].axvline(np.mean(article_sent_counts), color='red', linestyle='--',
#                      label=f'Mean: {np.mean(article_sent_counts):.1f}')
#     axes[1,0].legend()

#     # Summary sentence count
#     axes[1,1].hist(summary_sent_counts, bins=15, alpha=0.7, color='brown', edgecolor='black')
#     axes[1,1].set_title('Summary Sentence Count Distribution')
#     axes[1,1].set_xlabel('Number of Sentences')
#     axes[1,1].set_ylabel('Frequency')
#     axes[1,1].axvline(np.mean(summary_sent_counts), color='red', linestyle='--',
#                      label=f'Mean: {np.mean(summary_sent_counts):.1f}')
#     axes[1,1].legend()

#     # Correlation plot
#     axes[1,2].scatter(article_lengths, summary_lengths, alpha=0.5, s=1)
#     axes[1,2].set_title('Article vs Summary Length')
#     axes[1,2].set_xlabel('Article Length (Words)')
#     axes[1,2].set_ylabel('Summary Length (Words)')

#     # Add correlation coefficient
#     correlation = np.corrcoef(article_lengths, summary_lengths)[0,1]
#     axes[1,2].text(0.05, 0.95, f'Correlation: {correlation:.3f}',
#                   transform=axes[1,2].transAxes, verticalalignment='top',
#                   bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

#     plt.tight_layout()
#     plt.savefig(os.path.join(RESULTS_DIR, 'text_statistics.png'), dpi=300, bbox_inches='tight')
#     plt.close()

#     # Print statistics
#     stats = {
#         'article_length': {
#             'mean': np.mean(article_lengths),
#             'std': np.std(article_lengths),
#             'median': np.median(article_lengths),
#             'min': np.min(article_lengths),
#             'max': np.max(article_lengths)
#         },
#         'summary_length': {
#             'mean': np.mean(summary_lengths),
#             'std': np.std(summary_lengths),
#             'median': np.median(summary_lengths),
#             'min': np.min(summary_lengths),
#             'max': np.max(summary_lengths)
#         },
#         'compression_ratio': {
#             'mean': np.mean(ratios),
#             'std': np.std(ratios),
#             'median': np.median(ratios)
#         }
#     }

#     print(f"📈 Text Statistics:")
#     print(f"  Article length - Mean: {stats['article_length']['mean']:.1f}, "
#           f"Median: {stats['article_length']['median']:.1f}, "
#           f"Range: {stats['article_length']['min']}-{stats['article_length']['max']}")
#     print(f"  Summary length - Mean: {stats['summary_length']['mean']:.1f}, "
#           f"Median: {stats['summary_length']['median']:.1f}, "
#           f"Range: {stats['summary_length']['min']}-{stats['summary_length']['max']}")
#     print(f"  Compression ratio - Mean: {stats['compression_ratio']['mean']:.3f}, "
#           f"Median: {stats['compression_ratio']['median']:.3f}")

#     return stats

# def clean_text(text):
#     """Clean and normalize text"""
#     # Remove extra whitespace
#     text = re.sub(r'\s+', ' ', text)

#     # Remove special characters but keep punctuation
#     text = re.sub(r'[^\w\s\.\,\!\?\;\:\-\(\)]', '', text)

#     # Fix common issues
#     text = text.replace(' .', '.')
#     text = text.replace(' ,', ',')
#     text = text.replace('( ', '(')
#     text = text.replace(' )', ')')

#     return text.strip()

# def create_vocabulary_analysis(dataset, max_vocab_size=50000):
#     """Analyze vocabulary and create word frequency statistics"""
#     print("📚 Creating vocabulary analysis...")

#     ensure_dir(RESULTS_DIR)
#     ensure_dir(PROCESSED_DATA_DIR)  # Ensure this exists for vocabulary.json

#     # Collect text from training set
#     train_data = dataset['train']
#     sample_size = min(10000, len(train_data))  # Sample for faster analysis

#     # Fix: Convert numpy integers to Python integers
#     sample_indices = np.random.choice(len(train_data), sample_size, replace=False)
#     sample_indices = [int(idx) for idx in sample_indices]  # Convert to Python int

#     all_words = []
#     stopwords_en = set(stopwords.words('english'))

#     print(f"Analyzing vocabulary from {sample_size} samples...")

#     for idx in tqdm(sample_indices, desc="Processing vocabulary"):
#         article = train_data[idx]['article']
#         summary = train_data[idx]['highlights']

#         # Tokenize and clean
#         article_words = word_tokenize(clean_text(article.lower()))
#         summary_words = word_tokenize(clean_text(summary.lower()))

#         # Filter words
#         article_words = [w for w in article_words if w.isalpha() and w not in stopwords_en]
#         summary_words = [w for w in summary_words if w.isalpha() and w not in stopwords_en]

#         all_words.extend(article_words)
#         all_words.extend(summary_words)

#     # Create word frequency distribution
#     word_freq = Counter(all_words)
#     most_common = word_freq.most_common(max_vocab_size)

#     # Visualizations
#     fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

#     # Top 30 words
#     top_30_words = most_common[:30]
#     words, counts = zip(*top_30_words)

#     ax1.barh(range(len(words)), counts)
#     ax1.set_yticks(range(len(words)))
#     ax1.set_yticklabels(words)
#     ax1.set_title('Top 30 Most Frequent Words')
#     ax1.set_xlabel('Frequency')
#     ax1.invert_yaxis()

#     # Word frequency distribution (Zipf's law)
#     ranks = range(1, min(1000, len(most_common)) + 1)
#     freqs = [count for _, count in most_common[:len(ranks)]]

#     ax2.loglog(ranks, freqs, 'b-', alpha=0.7)
#     ax2.set_title('Word Frequency Distribution (Zipf\'s Law)')
#     ax2.set_xlabel('Rank')
#     ax2.set_ylabel('Frequency')
#     ax2.grid(True, alpha=0.3)

#     plt.tight_layout()
#     plt.savefig(os.path.join(RESULTS_DIR, 'vocabulary_analysis.png'), dpi=300, bbox_inches='tight')
#     plt.close()

#     # Save vocabulary
#     vocab_info = {
#         'total_words': len(all_words),
#         'unique_words': len(word_freq),
#         'most_common': most_common[:1000],  # Save top 1000
#         'vocabulary_size': len(word_freq)
#     }

#     with open(os.path.join(PROCESSED_DATA_DIR, 'vocabulary.json'), 'w') as f:
#         json.dump(vocab_info, f, indent=2)

#     print(f"  Total words processed: {len(all_words):,}")
#     print(f"  Unique words: {len(word_freq):,}")
#     print(f"  Most common word: '{most_common[0][0]}' ({most_common[0][1]} occurrences)")

#     return vocab_info

# def tokenize_and_preprocess(dataset, tokenizer_name='bert-base-uncased', max_samples=None):
#     """Tokenize text using a pre-trained tokenizer"""
#     print(f"🔤 Tokenizing text with {tokenizer_name}...")

#     try:
#         tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
#     except:
#         print(f"Warning: Could not load {tokenizer_name}, using basic preprocessing")
#         return preprocess_basic_tokenization(dataset, max_samples)

#     processed_splits = {}

#     for split_name, split_data in dataset.items():
#         print(f"Processing {split_name} split...")

#         # Limit samples for testing
#         if max_samples and split_name == 'train':
#             indices = list(range(min(max_samples, len(split_data))))  # Convert to list of Python ints
#         else:
#             indices = list(range(len(split_data)))  # Convert to list of Python ints

#         articles = []
#         summaries = []

#         for idx in tqdm(indices, desc=f"Tokenizing {split_name}"):
#             article = clean_text(split_data[idx]['article'])
#             summary = clean_text(split_data[idx]['highlights'])

#             # Tokenize article (truncate to model max length)
#             article_tokens = tokenizer(
#                 article,
#                 max_length=512,
#                 truncation=True,
#                 padding=False,
#                 return_tensors="pt"
#             )

#             # Tokenize summary
#             summary_tokens = tokenizer(
#                 summary,
#                 max_length=128,
#                 truncation=True,
#                 padding=False,
#                 return_tensors="pt"
#             )

#             articles.append({
#                 'input_ids': article_tokens['input_ids'].squeeze(),
#                 'attention_mask': article_tokens['attention_mask'].squeeze(),
#                 'text': article
#             })

#             summaries.append({
#                 'input_ids': summary_tokens['input_ids'].squeeze(),
#                 'attention_mask': summary_tokens['attention_mask'].squeeze(),
#                 'text': summary
#             })

#         processed_splits[split_name] = {
#             'articles': articles,
#             'summaries': summaries,
#             'count': len(articles)
#         }

#         print(f"✅ {split_name}: {len(articles)} samples processed")

#     return processed_splits, tokenizer

# def preprocess_basic_tokenization(dataset, max_samples=None):
#     """Basic text preprocessing without transformer tokenizer"""
#     print("🔤 Performing basic text preprocessing...")

#     processed_splits = {}

#     for split_name, split_data in dataset.items():
#         print(f"Processing {split_name} split...")

#         # Limit samples for testing
#         if max_samples and split_name == 'train':
#             indices = list(range(min(max_samples, len(split_data))))  # Convert to list of Python ints
#         else:
#             indices = list(range(len(split_data)))  # Convert to list of Python ints

#         articles = []
#         summaries = []

#         for idx in tqdm(indices, desc=f"Processing {split_name}"):
#             article = clean_text(split_data[idx]['article'])
#             summary = clean_text(split_data[idx]['highlights'])

#             # Basic tokenization
#             article_words = word_tokenize(article.lower())
#             summary_words = word_tokenize(summary.lower())

#             articles.append({
#                 'tokens': article_words,
#                 'text': article,
#                 'length': len(article_words)
#             })

#             summaries.append({
#                 'tokens': summary_words,
#                 'text': summary,
#                 'length': len(summary_words)
#             })

#         processed_splits[split_name] = {
#             'articles': articles,
#             'summaries': summaries,
#             'count': len(articles)
#         }

#         print(f"✅ {split_name}: {len(articles)} samples processed")

#     return processed_splits, None

# def save_processed_data(processed_splits, tokenizer, text_stats, vocab_info):
#     """Save processed CNN/DailyMail data"""
#     print("💾 Saving processed data...")

#     ensure_dir(PROCESSED_DATA_DIR)

#     # Save processed splits
#     for split_name, split_data in processed_splits.items():
#         with open(os.path.join(PROCESSED_DATA_DIR, f"{split_name}_processed.pkl"), 'wb') as f:
#             pickle.dump(split_data, f)

#     # Save tokenizer if available
#     if tokenizer:
#         tokenizer.save_pretrained(os.path.join(PROCESSED_DATA_DIR, 'tokenizer'))

#     # Save metadata
#     metadata = {
#         'dataset_name': 'CNN/DailyMail',
#         'task': 'text_summarization',
#         'splits': {name: data['count'] for name, data in processed_splits.items()},
#         'tokenizer': tokenizer.name_or_path if tokenizer else 'basic_tokenization',
#         'text_statistics': text_stats,
#         'vocabulary_info': {
#             'total_words': vocab_info['total_words'],
#             'unique_words': vocab_info['unique_words'],
#             'vocabulary_size': vocab_info['vocabulary_size']
#         },
#         'preprocessing_info': {
#             'max_article_length': 512,
#             'max_summary_length': 128,
#             'text_cleaning': True,
#             'truncation': True
#         }
#     }

#     with open(os.path.join(PROCESSED_DATA_DIR, 'metadata.json'), 'w') as f:
#         json.dump(metadata, f, indent=2)

#     print(f"✅ Data saved to {PROCESSED_DATA_DIR}")

#     # Print summary
#     total_samples = sum(data['count'] for data in processed_splits.values())
#     print(f"\n📋 Processing Summary:")
#     print(f"  Total samples: {total_samples:,}")
#     for split_name, data in processed_splits.items():
#         print(f"  {split_name}: {data['count']:,} samples")

# def create_sample_examples(processed_splits, num_examples=5):
#     """Create sample examples for visualization"""
#     print("📝 Creating sample examples...")

#     ensure_dir(RESULTS_DIR)

#     train_data = processed_splits['train']

#     # Fix: Convert numpy integers to Python integers
#     sample_indices = np.random.choice(len(train_data['articles']), num_examples, replace=False)
#     sample_indices = [int(idx) for idx in sample_indices]  # Convert to Python int

#     examples = []
#     for idx in sample_indices:
#         article = train_data['articles'][idx]
#         summary = train_data['summaries'][idx]

#         example = {
#             'index': idx,
#             'article': article['text'][:1000] + '...' if len(article['text']) > 1000 else article['text'],
#             'summary': summary['text'],
#             'article_length': len(article['text'].split()) if 'text' in article else article.get('length', 0),
#             'summary_length': len(summary['text'].split()) if 'text' in summary else summary.get('length', 0)
#         }
#         examples.append(example)

#     # Save examples as JSON for easy viewing
#     with open(os.path.join(RESULTS_DIR, 'sample_examples.json'), 'w') as f:
#         json.dump(examples, f, indent=2)

#     # Create a formatted text file
#     with open(os.path.join(RESULTS_DIR, 'sample_examples.txt'), 'w') as f:
#         f.write("CNN/DailyMail Dataset - Sample Examples\n")
#         f.write("=" * 50 + "\n\n")

#         for i, example in enumerate(examples, 1):
#             f.write(f"EXAMPLE {i}:\n")
#             f.write(f"Article Length: {example['article_length']} words\n")
#             f.write(f"Summary Length: {example['summary_length']} words\n")
#             f.write(f"Compression Ratio: {example['summary_length']/example['article_length']:.3f}\n\n")

#             f.write("ARTICLE:\n")
#             f.write(example['article'])
#             f.write("\n\nSUMMARY:\n")
#             f.write(example['summary'])
#             f.write("\n\n" + "-" * 50 + "\n\n")

#     print(f"✅ {num_examples} sample examples saved to {RESULTS_DIR}")
#     return examples

# def main(max_samples=None, tokenizer_name='bert-base-uncased', create_visualizations=True):
#     """Main CNN/DailyMail preprocessing function"""
#     print("🚀 Starting CNN/DailyMail Preprocessing...")
#     print("=" * 50)

#     # Create directories
#     ensure_dir(PROCESSED_DATA_DIR)
#     ensure_dir(RESULTS_DIR)

#     # Load dataset (will download if not exists)
#     dataset = load_cnn_dailymail_dataset()
#     if dataset is None:
#         print("❌ Failed to load dataset. Please check your internet connection.")
#         return False

#     try:
#         # Analyze text statistics
#         if create_visualizations:
#             print("\n📊 Analyzing text statistics...")
#             text_stats = analyze_text_statistics(dataset)

#             print("\n📚 Analyzing vocabulary...")
#             vocab_info = create_vocabulary_analysis(dataset)
#         else:
#             text_stats = {}
#             vocab_info = {'total_words': 0, 'unique_words': 0, 'vocabulary_size': 0}

#         # Tokenize and preprocess
#         print("\n🔤 Tokenizing and preprocessing...")
#         processed_splits, tokenizer = tokenize_and_preprocess(
#             dataset, tokenizer_name, max_samples
#         )

#         # Create sample examples
#         if create_visualizations and processed_splits:
#             create_sample_examples(processed_splits)

#         # Save processed data
#         if processed_splits:
#             save_processed_data(processed_splits, tokenizer, text_stats, vocab_info)
#             print("🎉 CNN/DailyMail preprocessing completed successfully!")
#             return True
#         else:
#             print("❌ No data was processed!")
#             return False

#     except Exception as e:
#         print(f"❌ Error during preprocessing: {e}")
#         import traceback
#         traceback.print_exc()
#         return False

# # Quick download function for immediate use
# def quick_download():
#     """Quick function to just download the dataset"""
#     print("🚀 Quick Download: CNN/DailyMail Dataset")
#     ensure_dir(os.path.dirname(RAW_DATA_DIR))
#     dataset = download_cnn_dailymail_dataset()
#     return dataset is not None

# if __name__ == "__main__":
#     # For testing, limit samples per split
#     # Remove max_samples parameter for full processing
#     success = main(
#         max_samples=1000,  # Limit for testing - set to None for full dataset
#         tokenizer_name='bert-base-uncased',
#         create_visualizations=True
#     )

#     if success:
#         print("\n✅ Preprocessing completed! You can now use the processed data for training.")
#         print(f"📁 Raw data: {RAW_DATA_DIR}")
#         print(f"📁 Processed data: {PROCESSED_DATA_DIR}")
#         print(f"📁 Visualizations: {RESULTS_DIR}")
#     else:
#         print("\n❌ Preprocessing failed! Check the error messages above.")

🚀 Starting CNN/DailyMail Preprocessing...
📂 Loading CNN/DailyMail dataset...
✅ Dataset loaded from disk with splits: ['train', 'validation', 'test']
  train: 287113 samples
  validation: 13368 samples
  test: 11490 samples

📊 Analyzing text statistics...
📊 Analyzing text statistics...
Analyzing 10000 samples from training set...


Analyzing samples: 100%|██████████| 10000/10000 [01:12<00:00, 137.79it/s]


📈 Text Statistics:
  Article length - Mean: 791.6, Median: 721.0, Range: 25-2161
  Summary length - Mean: 55.2, Median: 52.0, Range: 8-324
  Compression ratio - Mean: 0.085, Median: 0.073

📚 Analyzing vocabulary...
📚 Creating vocabulary analysis...
Analyzing vocabulary from 10000 samples...


Processing vocabulary: 100%|██████████| 10000/10000 [01:13<00:00, 136.80it/s]


  Total words processed: 3,980,084
  Unique words: 93,835
  Most common word: 'said' (46940 occurrences)

🔤 Tokenizing and preprocessing...
🔤 Tokenizing text with bert-base-uncased...


...

✅ validation: 13368 samples processed
Processing test split...


Tokenizing test: 100%|██████████| 11490/11490 [01:43<00:00, 110.90it/s]


✅ test: 11490 samples processed
📝 Creating sample examples...
✅ 5 sample examples saved to /content/drive/MyDrive/PaperSense-Intelligent-Document-Platform/results/visualizations/generation
💾 Saving processed data...
❌ Error during preprocessing: Object of type int64 is not JSON serializable


Traceback (most recent call last):
  File "/tmp/ipython-input-1572470490.py", line 572, in main
    save_processed_data(processed_splits, tokenizer, text_stats, vocab_info)
  File "/tmp/ipython-input-1572470490.py", line 472, in save_processed_data
    json.dump(metadata, f, indent=2)
  File "/usr/lib/python3.12/json/__init__.py", line 179, in dump
    for chunk in iterable:
                 ^^^^^^^^
  File "/usr/lib/python3.12/json/encoder.py", line 432, in _iterencode
    yield from _iterencode_dict(o, _current_indent_level)
  File "/usr/lib/python3.12/json/encoder.py", line 406, in _iterencode_dict
    yield from chunks
  File "/usr/lib/python3.12/json/encoder.py", line 406, in _iterencode_dict
    yield from chunks
  File "/usr/lib/python3.12/json/encoder.py", line 406, in _iterencode_dict
    yield from chunks
  File "/usr/lib/python3.12/json/encoder.py", line 439, in _iterencode
    o = _default(o)
        ^^^^^^^^^^^
  File "/usr/lib/python3.12/json/encoder.py", line 180, in def


❌ Preprocessing failed! Check the error messages above.


In [14]:
"""
CNN/DailyMail Dataset Preprocessing
Handles text summarization data preparation
"""

import os
import json
import pandas as pd
import numpy as np
import torch
from datasets import load_dataset, load_from_disk
import matplotlib.pyplot as plt
import seaborn as sns
from transformers import AutoTokenizer
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import re
from collections import Counter
from tqdm import tqdm
import pickle

# Download required NLTK data
nltk.download('punkt_tab', quiet=True)
nltk.download('stopwords', quiet=True)

# Project paths - FIXED
PROJECT_ROOT = "/content/drive/MyDrive/PaperSense-Intelligent-Document-Platform"
RAW_DATA_DIR = os.path.join(PROJECT_ROOT, "datasets", "raw", "cnn_dailymail")
PROCESSED_DATA_DIR = os.path.join(PROJECT_ROOT, "datasets", "processed", "cnn_dailymail")
RESULTS_DIR = os.path.join(PROJECT_ROOT, "results", "visualizations", "generation")

def ensure_dir(path):
    """Create directory if it doesn't exist"""
    os.makedirs(path, exist_ok=True)
    return path

def convert_numpy_types(obj):
    """Recursively convert numpy types to Python native types for JSON serialization"""
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, dict):
        return {key: convert_numpy_types(value) for key, value in obj.items()}
    elif isinstance(obj, list):
        return [convert_numpy_types(item) for item in obj]
    elif isinstance(obj, tuple):
        return tuple(convert_numpy_types(item) for item in obj)
    else:
        return obj

def download_cnn_dailymail_dataset():
    """Download and save CNN/DailyMail dataset"""
    print("📥 Downloading CNN/DailyMail dataset...")

    try:
        # Download dataset from Hugging Face
        dataset = load_dataset("cnn_dailymail", "3.0.0")

        # Create directory and save
        ensure_dir(os.path.dirname(RAW_DATA_DIR))
        dataset.save_to_disk(RAW_DATA_DIR)

        print(f"✅ Dataset downloaded and saved to: {RAW_DATA_DIR}")
        return dataset
    except Exception as e:
        print(f"❌ Error downloading dataset: {e}")
        return None

def load_cnn_dailymail_dataset():
    """Load CNN/DailyMail dataset from disk or download if not exists"""
    print("📂 Loading CNN/DailyMail dataset...")

    try:
        # Try to load from disk first
        if os.path.exists(RAW_DATA_DIR) and os.listdir(RAW_DATA_DIR):
            dataset = load_from_disk(RAW_DATA_DIR)
            print(f"✅ Dataset loaded from disk with splits: {list(dataset.keys())}")
        else:
            print("📥 Dataset not found on disk, downloading...")
            dataset = download_cnn_dailymail_dataset()
            if dataset is None:
                return None

        # Print basic info
        for split_name, split_data in dataset.items():
            print(f"  {split_name}: {len(split_data)} samples")

        return dataset
    except Exception as e:
        print(f"❌ Error loading dataset: {e}")
        print("🔄 Attempting to download dataset...")
        return download_cnn_dailymail_dataset()

def analyze_text_statistics(dataset):
    """Analyze text length and content statistics"""
    print("📊 Analyzing text statistics...")

    ensure_dir(RESULTS_DIR)

    # Collect statistics
    article_lengths = []
    summary_lengths = []
    article_sent_counts = []
    summary_sent_counts = []

    # Sample from train set for analysis
    train_data = dataset['train']
    sample_size = min(10000, len(train_data))  # Sample for faster analysis

    # Fix: Convert numpy integers to Python integers
    sample_indices = np.random.choice(len(train_data), sample_size, replace=False)
    sample_indices = [int(idx) for idx in sample_indices]  # Convert to Python int

    print(f"Analyzing {sample_size} samples from training set...")

    for idx in tqdm(sample_indices, desc="Analyzing samples"):
        article = train_data[idx]['article']
        summary = train_data[idx]['highlights']

        # Word counts
        article_words = len(word_tokenize(article))
        summary_words = len(word_tokenize(summary))
        article_lengths.append(article_words)
        summary_lengths.append(summary_words)

        # Sentence counts
        article_sents = len(sent_tokenize(article))
        summary_sents = len(sent_tokenize(summary))
        article_sent_counts.append(article_sents)
        summary_sent_counts.append(summary_sents)

    # Create visualizations
    fig, axes = plt.subplots(2, 3, figsize=(18, 10))

    # Article length distribution
    axes[0,0].hist(article_lengths, bins=50, alpha=0.7, color='blue', edgecolor='black')
    axes[0,0].set_title('Article Length Distribution (Words)')
    axes[0,0].set_xlabel('Number of Words')
    axes[0,0].set_ylabel('Frequency')
    axes[0,0].axvline(np.mean(article_lengths), color='red', linestyle='--',
                     label=f'Mean: {np.mean(article_lengths):.0f}')
    axes[0,0].legend()

    # Summary length distribution
    axes[0,1].hist(summary_lengths, bins=50, alpha=0.7, color='green', edgecolor='black')
    axes[0,1].set_title('Summary Length Distribution (Words)')
    axes[0,1].set_xlabel('Number of Words')
    axes[0,1].set_ylabel('Frequency')
    axes[0,1].axvline(np.mean(summary_lengths), color='red', linestyle='--',
                     label=f'Mean: {np.mean(summary_lengths):.0f}')
    axes[0,1].legend()

    # Length ratio distribution
    ratios = [s/a if a > 0 else 0 for a, s in zip(article_lengths, summary_lengths)]
    axes[0,2].hist(ratios, bins=50, alpha=0.7, color='orange', edgecolor='black')
    axes[0,2].set_title('Summary/Article Length Ratio')
    axes[0,2].set_xlabel('Ratio')
    axes[0,2].set_ylabel('Frequency')
    axes[0,2].axvline(np.mean(ratios), color='red', linestyle='--',
                     label=f'Mean: {np.mean(ratios):.3f}')
    axes[0,2].legend()

    # Article sentence count
    axes[1,0].hist(article_sent_counts, bins=30, alpha=0.7, color='purple', edgecolor='black')
    axes[1,0].set_title('Article Sentence Count Distribution')
    axes[1,0].set_xlabel('Number of Sentences')
    axes[1,0].set_ylabel('Frequency')
    axes[1,0].axvline(np.mean(article_sent_counts), color='red', linestyle='--',
                     label=f'Mean: {np.mean(article_sent_counts):.1f}')
    axes[1,0].legend()

    # Summary sentence count
    axes[1,1].hist(summary_sent_counts, bins=15, alpha=0.7, color='brown', edgecolor='black')
    axes[1,1].set_title('Summary Sentence Count Distribution')
    axes[1,1].set_xlabel('Number of Sentences')
    axes[1,1].set_ylabel('Frequency')
    axes[1,1].axvline(np.mean(summary_sent_counts), color='red', linestyle='--',
                     label=f'Mean: {np.mean(summary_sent_counts):.1f}')
    axes[1,1].legend()

    # Correlation plot
    axes[1,2].scatter(article_lengths, summary_lengths, alpha=0.5, s=1)
    axes[1,2].set_title('Article vs Summary Length')
    axes[1,2].set_xlabel('Article Length (Words)')
    axes[1,2].set_ylabel('Summary Length (Words)')

    # Add correlation coefficient
    correlation = np.corrcoef(article_lengths, summary_lengths)[0,1]
    axes[1,2].text(0.05, 0.95, f'Correlation: {correlation:.3f}',
                  transform=axes[1,2].transAxes, verticalalignment='top',
                  bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'text_statistics.png'), dpi=300, bbox_inches='tight')
    plt.close()

    # Print statistics - Convert numpy types to Python types
    stats = {
        'article_length': {
            'mean': float(np.mean(article_lengths)),
            'std': float(np.std(article_lengths)),
            'median': float(np.median(article_lengths)),
            'min': int(np.min(article_lengths)),
            'max': int(np.max(article_lengths))
        },
        'summary_length': {
            'mean': float(np.mean(summary_lengths)),
            'std': float(np.std(summary_lengths)),
            'median': float(np.median(summary_lengths)),
            'min': int(np.min(summary_lengths)),
            'max': int(np.max(summary_lengths))
        },
        'compression_ratio': {
            'mean': float(np.mean(ratios)),
            'std': float(np.std(ratios)),
            'median': float(np.median(ratios))
        }
    }

    print(f"📈 Text Statistics:")
    print(f"  Article length - Mean: {stats['article_length']['mean']:.1f}, "
          f"Median: {stats['article_length']['median']:.1f}, "
          f"Range: {stats['article_length']['min']}-{stats['article_length']['max']}")
    print(f"  Summary length - Mean: {stats['summary_length']['mean']:.1f}, "
          f"Median: {stats['summary_length']['median']:.1f}, "
          f"Range: {stats['summary_length']['min']}-{stats['summary_length']['max']}")
    print(f"  Compression ratio - Mean: {stats['compression_ratio']['mean']:.3f}, "
          f"Median: {stats['compression_ratio']['median']:.3f}")

    return stats

def clean_text(text):
    """Clean and normalize text"""
    # Remove extra whitespace
    text = re.sub(r'\s+', ' ', text)

    # Remove special characters but keep punctuation
    text = re.sub(r'[^\w\s\.\,\!\?\;\:\-\(\)]', '', text)

    # Fix common issues
    text = text.replace(' .', '.')
    text = text.replace(' ,', ',')
    text = text.replace('( ', '(')
    text = text.replace(' )', ')')

    return text.strip()

def create_vocabulary_analysis(dataset, max_vocab_size=50000):
    """Analyze vocabulary and create word frequency statistics"""
    print("📚 Creating vocabulary analysis...")

    ensure_dir(RESULTS_DIR)
    ensure_dir(PROCESSED_DATA_DIR)  # Ensure this exists for vocabulary.json

    # Collect text from training set
    train_data = dataset['train']
    sample_size = min(10000, len(train_data))  # Sample for faster analysis

    # Fix: Convert numpy integers to Python integers
    sample_indices = np.random.choice(len(train_data), sample_size, replace=False)
    sample_indices = [int(idx) for idx in sample_indices]  # Convert to Python int

    all_words = []
    stopwords_en = set(stopwords.words('english'))

    print(f"Analyzing vocabulary from {sample_size} samples...")

    for idx in tqdm(sample_indices, desc="Processing vocabulary"):
        article = train_data[idx]['article']
        summary = train_data[idx]['highlights']

        # Tokenize and clean
        article_words = word_tokenize(clean_text(article.lower()))
        summary_words = word_tokenize(clean_text(summary.lower()))

        # Filter words
        article_words = [w for w in article_words if w.isalpha() and w not in stopwords_en]
        summary_words = [w for w in summary_words if w.isalpha() and w not in stopwords_en]

        all_words.extend(article_words)
        all_words.extend(summary_words)

    # Create word frequency distribution
    word_freq = Counter(all_words)
    most_common = word_freq.most_common(max_vocab_size)

    # Visualizations
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

    # Top 30 words
    top_30_words = most_common[:30]
    words, counts = zip(*top_30_words)

    ax1.barh(range(len(words)), counts)
    ax1.set_yticks(range(len(words)))
    ax1.set_yticklabels(words)
    ax1.set_title('Top 30 Most Frequent Words')
    ax1.set_xlabel('Frequency')
    ax1.invert_yaxis()

    # Word frequency distribution (Zipf's law)
    ranks = range(1, min(1000, len(most_common)) + 1)
    freqs = [count for _, count in most_common[:len(ranks)]]

    ax2.loglog(ranks, freqs, 'b-', alpha=0.7)
    ax2.set_title('Word Frequency Distribution (Zipf\'s Law)')
    ax2.set_xlabel('Rank')
    ax2.set_ylabel('Frequency')
    ax2.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(os.path.join(RESULTS_DIR, 'vocabulary_analysis.png'), dpi=300, bbox_inches='tight')
    plt.close()

    # Save vocabulary - Convert to JSON-serializable format
    vocab_info = {
        'total_words': len(all_words),
        'unique_words': len(word_freq),
        'most_common': most_common[:1000],  # Save top 1000
        'vocabulary_size': len(word_freq)
    }

    # Convert numpy types to Python types
    vocab_info = convert_numpy_types(vocab_info)

    with open(os.path.join(PROCESSED_DATA_DIR, 'vocabulary.json'), 'w') as f:
        json.dump(vocab_info, f, indent=2)

    print(f"  Total words processed: {len(all_words):,}")
    print(f"  Unique words: {len(word_freq):,}")
    print(f"  Most common word: '{most_common[0][0]}' ({most_common[0][1]} occurrences)")

    return vocab_info

def tokenize_and_preprocess(dataset, tokenizer_name='bert-base-uncased', max_samples=None):
    """Tokenize text using a pre-trained tokenizer"""
    print(f"🔤 Tokenizing text with {tokenizer_name}...")

    try:
        tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    except:
        print(f"Warning: Could not load {tokenizer_name}, using basic preprocessing")
        return preprocess_basic_tokenization(dataset, max_samples)

    processed_splits = {}

    for split_name, split_data in dataset.items():
        print(f"Processing {split_name} split...")

        # Limit samples for testing
        if max_samples and split_name == 'train':
            indices = list(range(min(max_samples, len(split_data))))  # Convert to list of Python ints
        else:
            indices = list(range(len(split_data)))  # Convert to list of Python ints

        articles = []
        summaries = []

        for idx in tqdm(indices, desc=f"Tokenizing {split_name}"):
            article = clean_text(split_data[idx]['article'])
            summary = clean_text(split_data[idx]['highlights'])

            # Tokenize article (truncate to model max length)
            article_tokens = tokenizer(
                article,
                max_length=512,
                truncation=True,
                padding=False,
                return_tensors="pt"
            )

            # Tokenize summary
            summary_tokens = tokenizer(
                summary,
                max_length=128,
                truncation=True,
                padding=False,
                return_tensors="pt"
            )

            articles.append({
                'input_ids': article_tokens['input_ids'].squeeze(),
                'attention_mask': article_tokens['attention_mask'].squeeze(),
                'text': article
            })

            summaries.append({
                'input_ids': summary_tokens['input_ids'].squeeze(),
                'attention_mask': summary_tokens['attention_mask'].squeeze(),
                'text': summary
            })

        processed_splits[split_name] = {
            'articles': articles,
            'summaries': summaries,
            'count': len(articles)
        }

        print(f"✅ {split_name}: {len(articles)} samples processed")

    return processed_splits, tokenizer

def preprocess_basic_tokenization(dataset, max_samples=None):
    """Basic text preprocessing without transformer tokenizer"""
    print("🔤 Performing basic text preprocessing...")

    processed_splits = {}

    for split_name, split_data in dataset.items():
        print(f"Processing {split_name} split...")

        # Limit samples for testing
        if max_samples and split_name == 'train':
            indices = list(range(min(max_samples, len(split_data))))  # Convert to list of Python ints
        else:
            indices = list(range(len(split_data)))  # Convert to list of Python ints

        articles = []
        summaries = []

        for idx in tqdm(indices, desc=f"Processing {split_name}"):
            article = clean_text(split_data[idx]['article'])
            summary = clean_text(split_data[idx]['highlights'])

            # Basic tokenization
            article_words = word_tokenize(article.lower())
            summary_words = word_tokenize(summary.lower())

            articles.append({
                'tokens': article_words,
                'text': article,
                'length': len(article_words)
            })

            summaries.append({
                'tokens': summary_words,
                'text': summary,
                'length': len(summary_words)
            })

        processed_splits[split_name] = {
            'articles': articles,
            'summaries': summaries,
            'count': len(articles)
        }

        print(f"✅ {split_name}: {len(articles)} samples processed")

    return processed_splits, None

def save_processed_data(processed_splits, tokenizer, text_stats, vocab_info):
    """Save processed CNN/DailyMail data"""
    print("💾 Saving processed data...")

    ensure_dir(PROCESSED_DATA_DIR)

    # Save processed splits
    for split_name, split_data in processed_splits.items():
        with open(os.path.join(PROCESSED_DATA_DIR, f"{split_name}_processed.pkl"), 'wb') as f:
            pickle.dump(split_data, f)

    # Save tokenizer if available
    if tokenizer:
        tokenizer.save_pretrained(os.path.join(PROCESSED_DATA_DIR, 'tokenizer'))

    # Save metadata - Convert all numpy types to Python types
    metadata = {
        'dataset_name': 'CNN/DailyMail',
        'task': 'text_summarization',
        'splits': {name: data['count'] for name, data in processed_splits.items()},
        'tokenizer': tokenizer.name_or_path if tokenizer else 'basic_tokenization',
        'text_statistics': convert_numpy_types(text_stats),
        'vocabulary_info': {
            'total_words': vocab_info['total_words'],
            'unique_words': vocab_info['unique_words'],
            'vocabulary_size': vocab_info['vocabulary_size']
        },
        'preprocessing_info': {
            'max_article_length': 512,
            'max_summary_length': 128,
            'text_cleaning': True,
            'truncation': True
        }
    }

    # Ensure all data is JSON serializable
    metadata = convert_numpy_types(metadata)

    with open(os.path.join(PROCESSED_DATA_DIR, 'metadata.json'), 'w') as f:
        json.dump(metadata, f, indent=2)

    print(f"✅ Data saved to {PROCESSED_DATA_DIR}")

    # Print summary
    total_samples = sum(data['count'] for data in processed_splits.values())
    print(f"\n📋 Processing Summary:")
    print(f"  Total samples: {total_samples:,}")
    for split_name, data in processed_splits.items():
        print(f"  {split_name}: {data['count']:,} samples")

def create_sample_examples(processed_splits, num_examples=5):
    """Create sample examples for visualization"""
    print("📝 Creating sample examples...")

    ensure_dir(RESULTS_DIR)

    train_data = processed_splits['train']

    # Fix: Convert numpy integers to Python integers
    sample_indices = np.random.choice(len(train_data['articles']), num_examples, replace=False)
    sample_indices = [int(idx) for idx in sample_indices]  # Convert to Python int

    examples = []
    for idx in sample_indices:
        article = train_data['articles'][idx]
        summary = train_data['summaries'][idx]

        example = {
            'index': int(idx),  # Ensure Python int
            'article': article['text'][:1000] + '...' if len(article['text']) > 1000 else article['text'],
            'summary': summary['text'],
            'article_length': len(article['text'].split()) if 'text' in article else article.get('length', 0),
            'summary_length': len(summary['text'].split()) if 'text' in summary else summary.get('length', 0)
        }
        examples.append(example)

    # Convert to ensure JSON serializable
    examples = convert_numpy_types(examples)

    # Save examples as JSON for easy viewing
    with open(os.path.join(RESULTS_DIR, 'sample_examples.json'), 'w') as f:
        json.dump(examples, f, indent=2)

    # Create a formatted text file
    with open(os.path.join(RESULTS_DIR, 'sample_examples.txt'), 'w') as f:
        f.write("CNN/DailyMail Dataset - Sample Examples\n")
        f.write("=" * 50 + "\n\n")

        for i, example in enumerate(examples, 1):
            f.write(f"EXAMPLE {i}:\n")
            f.write(f"Article Length: {example['article_length']} words\n")
            f.write(f"Summary Length: {example['summary_length']} words\n")
            f.write(f"Compression Ratio: {example['summary_length']/example['article_length']:.3f}\n\n")

            f.write("ARTICLE:\n")
            f.write(example['article'])
            f.write("\n\nSUMMARY:\n")
            f.write(example['summary'])
            f.write("\n\n" + "-" * 50 + "\n\n")

    print(f"✅ {num_examples} sample examples saved to {RESULTS_DIR}")
    return examples

def main(max_samples=None, tokenizer_name='bert-base-uncased', create_visualizations=True):
    """Main CNN/DailyMail preprocessing function"""
    print("🚀 Starting CNN/DailyMail Preprocessing...")
    print("=" * 50)

    # Create directories
    ensure_dir(PROCESSED_DATA_DIR)
    ensure_dir(RESULTS_DIR)

    # Load dataset (will download if not exists)
    dataset = load_cnn_dailymail_dataset()
    if dataset is None:
        print("❌ Failed to load dataset. Please check your internet connection.")
        return False

    try:
        # Analyze text statistics
        if create_visualizations:
            print("\n📊 Analyzing text statistics...")
            text_stats = analyze_text_statistics(dataset)

            print("\n📚 Analyzing vocabulary...")
            vocab_info = create_vocabulary_analysis(dataset)
        else:
            text_stats = {}
            vocab_info = {'total_words': 0, 'unique_words': 0, 'vocabulary_size': 0}

        # Tokenize and preprocess
        print("\n🔤 Tokenizing and preprocessing...")
        processed_splits, tokenizer = tokenize_and_preprocess(
            dataset, tokenizer_name, max_samples
        )

        # Create sample examples
        if create_visualizations and processed_splits:
            create_sample_examples(processed_splits)

        # Save processed data
        if processed_splits:
            save_processed_data(processed_splits, tokenizer, text_stats, vocab_info)
            print("🎉 CNN/DailyMail preprocessing completed successfully!")
            return True
        else:
            print("❌ No data was processed!")
            return False

    except Exception as e:
        print(f"❌ Error during preprocessing: {e}")
        import traceback
        traceback.print_exc()
        return False

# Quick download function for immediate use
def quick_download():
    """Quick function to just download the dataset"""
    print("🚀 Quick Download: CNN/DailyMail Dataset")
    ensure_dir(os.path.dirname(RAW_DATA_DIR))
    dataset = download_cnn_dailymail_dataset()
    return dataset is not None

if __name__ == "__main__":
    # For testing, limit samples per split
    # Remove max_samples parameter for full processing
    success = main(
        max_samples=1000,  # Limit for testing - set to None for full dataset
        tokenizer_name='bert-base-uncased',
        create_visualizations=True
    )

    if success:
        print("\n✅ Preprocessing completed! You can now use the processed data for training.")
        print(f"📁 Raw data: {RAW_DATA_DIR}")
        print(f"📁 Processed data: {PROCESSED_DATA_DIR}")
        print(f"📁 Visualizations: {RESULTS_DIR}")
    else:
        print("\n❌ Preprocessing failed! Check the error messages above.")

🚀 Starting CNN/DailyMail Preprocessing...
📂 Loading CNN/DailyMail dataset...
✅ Dataset loaded from disk with splits: ['train', 'validation', 'test']
  train: 287113 samples
  validation: 13368 samples
  test: 11490 samples

📊 Analyzing text statistics...
📊 Analyzing text statistics...
Analyzing 10000 samples from training set...


Analyzing samples: 100%|██████████| 10000/10000 [01:16<00:00, 130.63it/s]


📈 Text Statistics:
  Article length - Mean: 782.3, Median: 718.0, Range: 49-2181
  Summary length - Mean: 54.5, Median: 51.0, Range: 7-426
  Compression ratio - Mean: 0.085, Median: 0.073

📚 Analyzing vocabulary...
📚 Creating vocabulary analysis...
Analyzing vocabulary from 10000 samples...


Processing vocabulary: 100%|██████████| 10000/10000 [01:02<00:00, 160.15it/s]


  Total words processed: 3,984,583
  Unique words: 94,106
  Most common word: 'said' (47196 occurrences)

🔤 Tokenizing and preprocessing...
🔤 Tokenizing text with bert-base-uncased...
Processing train split...


...

✅ train: 1000 samples processed
Processing validation split...


Tokenizing validation: 100%|██████████| 13368/13368 [01:47<00:00, 124.80it/s]


✅ validation: 13368 samples processed
Processing test split...


Tokenizing test: 100%|██████████| 11490/11490 [01:42<00:00, 111.64it/s]


✅ test: 11490 samples processed
📝 Creating sample examples...
✅ 5 sample examples saved to /content/drive/MyDrive/PaperSense-Intelligent-Document-Platform/results/visualizations/generation
💾 Saving processed data...
✅ Data saved to /content/drive/MyDrive/PaperSense-Intelligent-Document-Platform/datasets/processed/cnn_dailymail

📋 Processing Summary:
  Total samples: 25,858
  train: 1,000 samples
  validation: 13,368 samples
  test: 11,490 samples
🎉 CNN/DailyMail preprocessing completed successfully!

✅ Preprocessing completed! You can now use the processed data for training.
📁 Raw data: /content/drive/MyDrive/PaperSense-Intelligent-Document-Platform/datasets/raw/cnn_dailymail
📁 Processed data: /content/drive/MyDrive/PaperSense-Intelligent-Document-Platform/datasets/processed/cnn_dailymail
📁 Visualizations: /content/drive/MyDrive/PaperSense-Intelligent-Document-Platform/results/visualizations/generation
